In [1]:
import mysql.connector
import pandas as pd

In [2]:
connection=mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='cric_sheet'
)

In [3]:
cursor=connection.cursor()

In [4]:
import pandas as pd

query = '''
WITH batting_stats AS (
    SELECT 
        batsman,
        COUNT(*) AS balls_faced,
        SUM(runs_batsman) AS total_runs
    FROM odi_match_deliveries
    WHERE batsman IS NOT NULL
    GROUP BY batsman
),
dismissals AS (
    SELECT 
        batsman,
        COUNT(*) AS times_out
    FROM odi_match_deliveries
    WHERE player_out IS NOT NULL
    GROUP BY batsman
)
SELECT 
    b.batsman,
    b.total_runs,
    b.balls_faced,
    d.times_out,
    ROUND(b.total_runs / NULLIF(d.times_out, 0), 2) AS batting_average,
    ROUND((b.total_runs / NULLIF(b.balls_faced, 0)) * 100, 2) AS strike_rate
FROM batting_stats b
LEFT JOIN dismissals d
    ON b.batsman = d.batsman
WHERE b.balls_faced >= 30
ORDER BY batting_average DESC;
'''

# Execute and fetch results into DataFrame
df_batting_stats = pd.read_sql(query, con=connection)

# Preview first 10 rows
print(df_batting_stats.head(10))


C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\2525519651.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_batting_stats = pd.read_sql(query, con=connection)


        batsman  total_runs  balls_faced  times_out  batting_average  \
0  CR Pagydyala       824.0         1228        1.0            824.0   
1       A Rawat       612.0          752        1.0            612.0   
2   MP Breetzke       932.0          952        2.0            466.0   
3       A Minny       424.0          520        1.0            424.0   
4      IA Karim       784.0          948        2.0            392.0   
5  AL Kandappah       388.0          636        1.0            388.0   
6   Aamer Yamin       380.0          404        1.0            380.0   
7      R Slater       360.0          412        1.0            360.0   
8        G Voll       692.0          656        2.0            346.0   
9   Nurul Hasan       660.0          724        2.0            330.0   

   strike_rate  
0        67.10  
1        81.38  
2        97.90  
3        81.54  
4        82.70  
5        61.01  
6        94.06  
7        87.38  
8       105.49  
9        91.16  


In [5]:
query2 = '''
WITH bowler_death_overs AS (
    SELECT 
        bowler,
        COUNT(*) AS balls_bowled,
        SUM(runs_total) AS runs_given
    FROM odi_match_deliveries
    WHERE `over` BETWEEN 41 AND 50
    GROUP BY bowler
)
SELECT 
    bowler,
    balls_bowled,
    runs_given,
    ROUND(runs_given / (balls_bowled / 6.0), 2) AS economy_rate
FROM bowler_death_overs
WHERE balls_bowled >= 30
ORDER BY economy_rate;
'''

df_pressure_perf = pd.read_sql(query2, con=connection)
print(df_pressure_perf.head())


C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\144684100.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pressure_perf = pd.read_sql(query2, con=connection)


        bowler  balls_bowled  runs_given  economy_rate
0   Sania Khan            48         4.0          0.50
1     AY Karim            48         8.0          1.00
2     S Horley            48         8.0          1.00
3     SA Fritz           124        24.0          1.16
4  B Siddharth            52        12.0          1.38


In [6]:
query3='''
with bowler_stats as (
select
bowler,
count(*) as balls_bowled,
sum(case when wicket_kind is not null then 1 else 0 end) as wickets
from odi_match_deliveries
group by bowler
)

select 
bowler,
balls_bowled,
wickets,
round(balls_bowled/nullif(wickets,0),2) as strike_rate
from bowler_stats
where balls_bowled >=100
order by strike_rate;
'''

bowler_sr_df=pd.read_sql(query3,con=connection)
print(bowler_sr_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1295807178.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bowler_sr_df=pd.read_sql(query3,con=connection)


         bowler  balls_bowled  wickets  strike_rate
0   Aaron Jones           800      0.0          NaN
1    Abul Hasan           884      0.0          NaN
2   ADJ Pitcher           152      0.0          NaN
3  ADS Fletcher           112      0.0          NaN
4  AGC Vimukthi           248      0.0          NaN


In [7]:
query4='''

WITH team_results AS (
    SELECT
        match_id,
        winner,
        CASE 
            WHEN toss_winner = team1 AND toss_decision = 'bat' THEN team1
            WHEN toss_winner = team2 AND toss_decision = 'bat' THEN team2
            WHEN toss_winner = team1 AND toss_decision = 'field' THEN team2
            WHEN toss_winner = team2 AND toss_decision = 'field' THEN team1
        END AS batting_first
    FROM odi_match_deliveries
    GROUP BY match_id, winner, toss_winner, toss_decision, team1, team2
)
SELECT 
    batting_first,
    COUNT(*) AS matches_played,
    SUM(CASE WHEN batting_first = winner THEN 1 ELSE 0 END) AS wins_batting_first,
    ROUND(SUM(CASE WHEN batting_first = winner THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS win_pct_batting_first
FROM team_results
GROUP BY batting_first
ORDER BY win_pct_batting_first DESC;
'''
team_results_df=pd.read_sql(query4,con=connection)
print(team_results_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\4202354857.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  team_results_df=pd.read_sql(query4,con=connection)


  batting_first  matches_played  wins_batting_first  win_pct_batting_first
0       Asia XI               3                 3.0                 100.00
1      Thailand               7                 6.0                  85.71
2     Australia             318               206.0                  64.78
3  South Africa             250               145.0                  58.00
4         India             300               169.0                  56.33


In [8]:
query4='''
with toss_impact as(
select match_id, max(toss_winner) as toss_winner,
max(toss_decision) as toss_decision,
max(winner) as winner
from odi_match_deliveries
group by match_id
)

select toss_decision,
count(*) as matches_played,
SUM(CASE WHEN toss_winner = winner THEN 1 ELSE 0 END) AS wins_when_followed,
ROUND(SUM(CASE WHEN toss_winner = winner THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS win_pct
from toss_impact
group by toss_decision
order by win_pct desc;
'''

toss_impact_df=pd.read_sql(query4,con=connection)
print(toss_impact_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\2610170743.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  toss_impact_df=pd.read_sql(query4,con=connection)


  toss_decision  matches_played  wins_when_followed  win_pct
0         field            1513               746.0    49.31
1           bat            1473               675.0    45.82


In [9]:
query5='''
select wicket_kind,count(*)as dismissals
from odi_match_deliveries
where wicket_kind is not null
group by wicket_kind
order by dismissals desc;
'''
dismissals_df=pd.read_sql(query5,con=connection)
print(dismissals_df.head(15))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1374302050.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dismissals_df=pd.read_sql(query5,con=connection)


             wicket_kind  dismissals
0                 caught       24028
1                 bowled        8255
2                    lbw        4972
3                run out        3328
4      caught and bowled        1378
5                stumped        1178
6           retired hurt          64
7             hit wicket          35
8  obstructing the field           6
9        retired not out           4


In [10]:
query_odi1 = '''
with phase_runs as (
select match_id,
case
when `over` between 1 and 10 then 'powerplay(1-10)'
when `over` between 11 and 40 then 'middle(11-40)'
when `over` between 41 and 50 then 'death(41-50)'
end as phase,
sum(runs_total) as runs_in_phase
from odi_match_deliveries
group by match_id,phase
)

select
phase,
round(avg(runs_in_phase),2) as avg_runs_scored
from phase_runs
group by phase
order by
case 
when phase='powerplay(1-10)' then 1
when phase='middle(11-40)' then 2
when phase='death(41-50)' then 3
end;
'''
odi1_df=pd.read_sql(query_odi1,con=connection)
print(odi1_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1838124335.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi1_df=pd.read_sql(query_odi1,con=connection)


             phase  avg_runs_scored
0             None            29.62
1  powerplay(1-10)           371.67
2    middle(11-40)          1053.76
3     death(41-50)           346.92


In [11]:
query_odi2='''
with player_match_scores as (
select match_id, batsman, sum(runs_batsman) as runs_in_match
from odi_match_deliveries
where batsman is not null
group by match_id,batsman
)

select batsman,
round(avg(runs_in_match),2) as avg_runs,
round(stddev(runs_in_match),2) as stddev_runs
from player_match_scores
group by batsman
having count(*) >=5
order by stddev_runs asc, avg_runs desc
limit 20;'''

odi2_df=pd.read_sql(query_odi2,con=connection)
print(odi2_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\2916786034.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi2_df=pd.read_sql(query_odi2,con=connection)


           batsman  avg_runs  stddev_runs
0        L Mabhero      0.80         1.60
1       PT Collins      2.40         1.96
2      JT Brassell      1.60         1.96
3   Ebadat Hossain      2.00         2.00
4  Al-Amin Hossain      2.67         2.98


In [12]:
query_odi3='''
select
team1 as team,
count(distinct match_id) as matches
from odi_match_deliveries
group by team1
order by matches desc
limit 10;
'''

odi3_df=pd.read_sql(query_odi3,con=connection)
display(odi3_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\397252511.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi3_df=pd.read_sql(query_odi3,con=connection)


,team,matches
0,India,326
1,Australia,324
2,England,318
3,New Zealand,252
4,South Africa,250
5,Sri Lanka,242
6,Bangladesh,225
7,West Indies,211
8,Pakistan,183
9,Zimbabwe,147


In [13]:
query_odi4='''
select
venue,count(distinct match_id) as matches
from odi_match_deliveries
group by venue
order by matches desc'''

odi4_df=pd.read_sql(query_odi4,con=connection)
display(odi4_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\3366502514.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi4_df=pd.read_sql(query_odi4,con=connection)


,venue,matches
0,Harare Sports Club,122
1,Shere Bangla National Stadium,85
2,Dubai International Cricket Stadium,56
3,Rangiri Dambulla International Stadium,51
4,R Premadasa Stadium,49
...,...,...
365,Wanderers Cricket Ground,1
366,"Western Australia Cricket Association Ground, ...",1
367,"Westpac Park, Hamilton",1
368,"Westpac Stadium, Wellington",1


In [14]:
query_odi5='''
select `over`,round(avg(runs_total),2) as avg_runs
from odi_match_deliveries
group by `over`
order by `over`
'''

odi5_df=pd.read_sql(query_odi5,con=connection)
display(odi5_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1295579987.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi5_df=pd.read_sql(query_odi5,con=connection)


,over,avg_runs
0,0,0.59
1,1,0.68
2,2,0.71
3,3,0.76
4,4,0.78
5,5,0.80
6,6,0.78
7,7,0.80
8,8,0.77
9,9,0.78


In [15]:
query_odi6='''
select 
bowler,
count(*) as balls_bowled,
sum(runs_total) as runs_given,
round(sum(runs_total)/(count(*)/6.0),2) as economy_rate
from odi_match_deliveries
where `over` between 11 and 40
group by bowler
having balls_bowled >=60
order by economy_rate 
limit 10;
'''
odi6_df=pd.read_sql(query_odi6,con=connection)
display(odi6_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1853928260.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odi6_df=pd.read_sql(query_odi6,con=connection)


,bowler,balls_bowled,runs_given,economy_rate
0,APC Kilowan,124,12.0,0.58
1,N David,316,68.0,1.29
2,M Shezi,148,32.0,1.30
3,C van der Westhuizen,200,52.0,1.56
4,RL Bhudia,144,44.0,1.83
5,NJ Shaw,440,136.0,1.85
6,GH O'Brien,120,40.0,2.00
7,SA Fritz,376,128.0,2.04
8,JI Rodrigues,132,48.0,2.18
9,SL Pujare,488,184.0,2.26


In [16]:
query_t20_1='''
with powerplay_runs as (select 
match_id,batting_team,round(sum(runs_total),2) as runs_scored
from t20_match_deliveries
where `over` between 1 and 6
group by match_id,batting_team)

select batting_team, round(avg(runs_scored),2) as avg_powerplay_runs
from powerplay_runs
group by batting_team
order by avg_powerplay_runs desc;
'''
t20_1_df=pd.read_sql(query_t20_1,con=connection)
display(t20_1_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1480471331.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t20_1_df=pd.read_sql(query_t20_1,con=connection)


,batting_team,avg_powerplay_runs
0,Romania,159.39
1,Saudi Arabia,146.17
2,Hungary,145.35
3,Australia,143.53
4,England,142.13


In [17]:
query_t20_2='''
with batsmen_strike_rate as (
select batsman, count(*) as balls_faced,sum(runs_batsman) as runs_scored
from t20_match_deliveries
group by batsman)

select batsman,round(runs_scored * 100.0/balls_faced,2) as strike_rate
from batsmen_strike_rate
where balls_faced >=200
order by strike_rate desc
limit 10;
'''

t20_2_df=pd.read_sql(query_t20_2,con=connection)
display(t20_2_df.head(12))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\3650676850.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t20_2_df=pd.read_sql(query_t20_2,con=connection)


,batsman,strike_rate
0,Manan Bashir,264.65
1,Muhammad Fahad,234.44
2,Taranjit Singh,201.37
3,Mansoor Safi,198.57
4,JL Smith,191.18
5,A Donovan,187.84
6,Mehboob Ali,186.13
7,Abhishek Sharma,185.12
8,Sahil Chauhan,184.62
9,ED Silva,178.49


In [18]:
query_t20_3='''
with bowler_economy_rate as (
select
bowler,count(*) as balls_bowled,sum(runs_total) as runs_given
from t20_match_deliveries
where `over` between 16 and 20
group by bowler 
)

select bowler,
round((runs_given/(balls_bowled/6)),2) as economy_rate
from bowler_economy_rate
where balls_bowled >100
order by economy_rate
limit 10
'''

t20_3_df=pd.read_sql(query_t20_3,con=connection)
display(t20_3_df.head(10))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\862071598.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t20_3_df=pd.read_sql(query_t20_3,con=connection)


,bowler,economy_rate
0,I Chathurani,2.07
1,YM Murrure,2.12
2,L Anyait,2.73
3,O Mondlane,3.07
4,Ashwini Balaji,3.19
5,D Koralalage,3.19
6,SAN Maypriani,3.26
7,NW Sariani,3.32
8,Uswa Syed,3.38
9,L Jie,3.41


In [19]:
query_t20_4='''
with bowler_over_runs as (
select match_id,bowler,`over`,sum(runs_total) as runs
from t20_match_deliveries
group by match_id,bowler,`over`
)
 
select 
bowler,
avg(runs) as avg_runs_per_over,
stddev(runs) as consistency_score,
count(*) as overs_bowled
from bowler_over_runs
group by bowler
having overs_bowled >=30
order by consistency_score
limit 10
'''

t20_4_df=pd.read_sql(query_t20_4,con=connection)
display(t20_4_df.head(15))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\341851739.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t20_4_df=pd.read_sql(query_t20_4,con=connection)


,bowler,avg_runs_per_over,consistency_score,overs_bowled
0,DAL Paramitha,9.1000,5.166237,30
1,L Jie,10.9412,5.291176,34
2,Han Lili,10.9091,5.418060,33
3,SAN Maypriani,8.6000,5.678028,45
4,ZL Thoon,10.5000,5.760859,32
5,EU Kejarukua,12.4286,6.424999,56
6,RV Hawkins,13.0000,6.442049,36
7,I Chathurani,9.2000,6.615134,30
8,R Shimwamana,9.8136,6.760991,59
9,Wu Juan,10.5000,6.873864,32


In [20]:
query_t20_5='''
with dot_balls as (
select
bowler,
count(*) as balls_bowled,
sum(runs_total=0) as dot_balls
from t20_match_deliveries
group by bowler
)

select bowler,
round(dot_balls/balls_bowled,2) as dot_balls_percentage
from dot_balls
group by bowler
order by dot_balls_percentage desc
'''

t20_5_df=pd.read_sql(query_t20_5, con=connection)
display(t20_5_df.head(10))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\4193561286.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t20_5_df=pd.read_sql(query_t20_5, con=connection)


,bowler,dot_balls_percentage
0,A Hewage,1.00
1,C Kauripeke,1.00
2,TJ Ramphoma,1.00
3,P Chakraborty,0.92
4,S Khatiwada,0.92
5,Anjali Chand,0.91
6,AS Taai,0.88
7,Apsari Begam,0.86
8,M Salum,0.85
9,Abhijeet Ahuja,0.83


In [21]:
query_test_1='''
select 
batsman,
cast(sum(runs_batsman) as signed) as total_runs
from test_match_deliveries
group by batsman
order by total_runs desc
limit 10
'''
test_1_df=pd.read_sql(query_test_1,con=connection)
display(test_1_df.head(10))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\328580145.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_1_df=pd.read_sql(query_test_1,con=connection)


,batsman,total_runs
0,JE Root,26518
1,AN Cook,24944
2,SPD Smith,20954
3,KS Williamson,18552
4,V Kohli,18460
5,HM Amla,18292
6,DA Warner,17572
7,KC Sangakkara,16978
8,AB de Villiers,16364
9,AD Mathews,16146


In [22]:
query_test_2='''
select bowler, count(player_out) as wickets
from test_match_deliveries
where player_out is not null
group by bowler
order by wickets desc
limit 10
'''

test_2_df=pd.read_sql(query_test_2,con=connection)
display(test_2_df.head(10))

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\211865803.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_2_df=pd.read_sql(query_test_2,con=connection)


,bowler,wickets
0,JM Anderson,705
1,SCJ Broad,616
2,NM Lyon,583
3,R Ashwin,553
4,DW Steyn,439
5,MA Starc,418
6,HMRKB Herath,416
7,TG Southee,400
8,K Rabada,346
9,RA Jadeja,328


In [24]:
query_test_3='''
with batsman_runs as(
select match_id,innings,batsman,sum(runs_batsman) as total_runs
from test_match_deliveries
group by match_id,innings,batsman)

select batsman, count(total_runs) as centuries
from batsman_runs
where total_runs>=100
group by batsman
order by centuries desc

'''

test_3_df=pd.read_sql(query_test_3,con=connection)
display(test_3_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\3002284708.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_3_df=pd.read_sql(query_test_3,con=connection)


,batsman,centuries
0,JE Root,103
1,AN Cook,90
2,SPD Smith,79
3,KS Williamson,70
4,HM Amla,67


In [27]:
query_test_4='''
with batsman_full_data as (
    select 
        match_id,
        innings,
        batsman,
        sum(runs_batsman) as total_runs
    from test_match_deliveries
    group by match_id, innings, batsman
)
select bfd.batsman, bfd.total_runs as highest_score
from batsman_full_data bfd
where bfd.total_runs = (
    select max(total_runs) from batsman_full_data
);

'''

test_4_df=pd.read_sql(query_test_4,con=connection)
display(test_4_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\3722721828.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_4_df=pd.read_sql(query_test_4,con=connection)


,batsman,highest_score
0,BC Lara,800.0


In [28]:
query_test_5='''
with bowler_wickets as (
    select match_id, innings, bowler,
           count(player_out) as wickets
    from test_match_deliveries
    where player_out is not null
    group by match_id, innings, bowler
),
five_wicket_hauls as (
    select bowler
    from bowler_wickets
    where wickets >= 5
)
select bowler, count(*) as five_wicket_hauls
from five_wicket_hauls
group by bowler
order by five_wicket_hauls desc
limit 10;
'''

test_5_df=pd.read_sql(query_test_5,con=connection)
display(test_5_df.head())

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\829675379.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_5_df=pd.read_sql(query_test_5,con=connection)


,bowler,five_wicket_hauls
0,R Ashwin,41
1,HMRKB Herath,36
2,JM Anderson,35
3,DW Steyn,28
4,NM Lyon,26


In [29]:
query_ipl_1='''
select batsman, sum(runs_batsman) as total_runs
from ipl_match_deliveries
group by batsman
order by total_runs desc
limit 10;
'''
ipl_1_df=pd.read_sql(query_ipl_1,con=connection)
display(ipl_1_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\2043921834.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ipl_1_df=pd.read_sql(query_ipl_1,con=connection)


,batsman,total_runs
0,V Kohli,17342.0
1,RG Sharma,14096.0
2,S Dhawan,13538.0
3,DA Warner,13134.0
4,SK Raina,11072.0
5,MS Dhoni,10878.0
6,KL Rahul,10470.0
7,AB de Villiers,10362.0
8,AM Rahane,10064.0
9,CH Gayle,9994.0


In [30]:
query_ipl_2='''
select bowler, count(player_out) as wickets
from ipl_match_deliveries
where player_out is not null
group by bowler
order by wickets desc
limit 10;
'''
ipl_2_df=pd.read_sql(query_ipl_2,con=connection)
display(ipl_2_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\3502185163.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ipl_2_df=pd.read_sql(query_ipl_2,con=connection)


,bowler,wickets
0,YS Chahal,229
1,B Kumar,213
2,SP Narine,212
3,DJ Bravo,207
4,R Ashwin,205
5,JJ Bumrah,203
6,PP Chawla,201
7,SL Malinga,188
8,A Mishra,183
9,RA Jadeja,179


In [31]:
query_ipl_3='''
select batsman, count(*) as sixes
from ipl_match_deliveries
where runs_batsman = 6
group by batsman
order by sixes desc
limit 10;
'''
ipl_3_df=pd.read_sql(query_ipl_3,con=connection)
display(ipl_3_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\1395594602.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ipl_3_df=pd.read_sql(query_ipl_3,con=connection)


,batsman,sixes
0,CH Gayle,718
1,RG Sharma,606
2,V Kohli,584
3,MS Dhoni,528
4,AB de Villiers,506
5,DA Warner,472
6,KA Pollard,448
7,AD Russell,446
8,SV Samson,438
9,KL Rahul,416


In [32]:
query_ipl_4='''
select bowler,
       sum(runs_total) as runs_conceded,
       count(*)/6 as overs_bowled,
       round(sum(runs_total) / (count(*)/6), 2) as economy_rate
from ipl_match_deliveries
group by bowler
having overs_bowled >= 50
order by economy_rate asc
limit 10;
'''
ipl_4_df=pd.read_sql(query_ipl_4,con=connection)
display(ipl_4_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\552430010.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ipl_4_df=pd.read_sql(query_ipl_4,con=connection)


,bowler,runs_conceded,overs_bowled,economy_rate
0,Sohail Tanvir,550.0,88.3333,6.23
1,A Chandila,490.0,78.0000,6.28
2,FH Edwards,320.0,50.0000,6.40
3,SMSM Senanayake,422.0,65.0000,6.49
4,SM Pollock,614.0,93.3333,6.58
5,A Kumble,2178.0,327.6667,6.65
6,GD McGrath,732.0,109.6667,6.67
7,M Muralitharan,3530.0,527.0000,6.70
8,IS Sodhi,408.0,60.6667,6.73
9,J Yadav,894.0,132.6667,6.74


In [35]:
query_ipl_5='''
select bowler,
       player_out as batsman,
       count(*) as dismissals
from ipl_match_deliveries
where player_out is not null
group by bowler, player_out
order by dismissals desc
limit 10;
;
'''
ipl_5_df=pd.read_sql(query_ipl_5,con=connection)
display(ipl_5_df)

C:\Users\tamiz\AppData\Local\Temp\ipykernel_4252\650522021.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ipl_5_df=pd.read_sql(query_ipl_5,con=connection)


,bowler,batsman,dismissals
0,SP Narine,RG Sharma,8
1,YS Chahal,MA Agarwal,7
2,MM Sharma,AT Rayudu,7
3,RA Jadeja,GJ Maxwell,7
4,A Mishra,RG Sharma,7
5,Z Khan,MS Dhoni,7
6,B Kumar,AM Rahane,7
7,R Ashwin,RV Uthappa,7
8,JJ Bumrah,RR Pant,7
9,Sandeep Sharma,V Kohli,7
